## __TOKENIZATION__

Tokenization is at the heart of much weirdness of LLMs. Do not brush it off.
- Why can't LLM spell words? __Tokenization__.
- Why can't LLM do super simple string processing tasks like reversing a string? __Tokenization__.
- Why is LLM worse at non-English languages (e.g. Japanese)? __Tokenization__.
- Why is LLM bad at simple arithmetic? __Tokenization__.
- Why did GPT-2 have more than necessary trouble coding in Python? __Tokenization__.
- Why did my LLM abruptly halt when it sees the string "<|endoftext|>*? __Tokenization__.
- What is this weird waring I get about a "trailing whitespace"? __Tokenization__.
- Why the LLM break it I ask it about "SolidGoldMagikarp"? __Tokenization__.
- Why should I prefer to use VAML over JSON with LLMs? __Tokenization__.
- Why is LLM not actually end-to-end language modeling? __Tokenization__.
- What is the real root of suffering? __Tokenization__.

look at tiktokenizer.vercel.app

#### Here's the overarching plan (take strings and feed them into language models):
1. Tokenize strings into some integers in some fixed vocabulary
2. Use those integers to make a lookup into a lookup table as inputs
3. Feed those vectors into the transformer as input

This won't be simple as we want to support multiple lanugages (Korean, etc.) and we want to support special characters (emojis, etc.)

What is Unicode

a mapping of around 150k characters

can be used using ord(x) for a character x in a string

Disadvantages
- too many inputs (150k)
- a constantly updating input (unstable)

In [1]:
[ord(x) for x in "안녕하세요 👋 (hello in Korean!)"]

[50504,
 45397,
 54616,
 49464,
 50836,
 32,
 128075,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

Unicode defines three types of Encodings. UTF-8, UTF-16, UTF-32

UTF-8 is the most common type of encoding, translating characters by using 1 to 4 bytes and accounts for 98% of web pages. Also backwards compatible to ASCII text.

In [2]:
list("안녕하세요 👋 (hello in Korean!)".encode("utf-8")) # returns in bytes wihtout list

[236,
 149,
 136,
 235,
 133,
 149,
 237,
 149,
 152,
 236,
 132,
 184,
 236,
 154,
 148,
 32,
 240,
 159,
 145,
 139,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

#### We don't want to use raw utf 8 encoding, because 
1. **UTF-8 bytes by itself only ranges from 0 to 255. When representing certain characters, sequence length can start exploding.**<br>
    - For example, when representing a korean characters we may use up to 3 bytes just for a single character. 
    - This could make a whole word explode to 10-20 bytes
    - Text sequences become an even larger token sequence
2. **Long sequences are a big problem because we cannot effectively take advantage of the context length**
    - When we optimize our model, we will reach an ideal maximum context length for our transformer. 
    - But if each word takes 3-10 tokens instead of 1-2, then we cannot truly use as much context (even though we are taking in the same number of tokens)
    - In byte level encoding we might only get like 300 words per 2k tokens vs 1k words in a better tokenizer
    - Model learns and performs worse since it can't "see" as much text
3. **If we want to use same context, attention is too expensive since sequences are so long**
4. **Solution: Byte Pair Encoding**

Byte pair encoding: An algorithm that finds the most recurring pair of two tokens and replaces that with a singular token

Ex:
aaabdaaabac

Z = aa

ZabdZabac

Y = ab

ZYdZYac

X = ZY

XdXac

^ this cannot be compressed further by byte pair encoding

In [3]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."

tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert from bytes to list of ints from 0 - 255 (the map part is actually redundant)

print("---")
print(text, "\nlength: ", len(text))
print("---")
print(tokens, "\nlength: ", len(tokens))

---
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception. 
length:  533
---
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 1

Pure utf coding brought 533 -> 616

In [4]:
def max_pairs(tokens):
    counts = {}
    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

counts = max_pairs(tokens)
res = sorted(((v, k) for k, v in counts.items()), reverse=True)
print(res)


[(20, (101, 32)), (15, (240, 159)), (12, (226, 128)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (116, 104)), (7, (159, 135)), (7, (159, 133)), (7, (97, 114)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (116, 32)), (6, (114, 32)), (6, (111, 114)), (6, (110, 103)), (6, (110, 100)), (6, (109, 101)), (6, (104, 101)), (6, (101, 114)), (6, (32, 105)), (5, (117, 115)), (5, (115, 116)), (5, (110, 32)), (5, (100, 101)), (5, (44, 32)), (5, (32, 115)), (4, (116, 105)), (4, (116, 101)), (4, (115, 44)), (4, (114, 105)), (4, (111, 117)), (4, (111, 100)), (4, (110, 116)), (4, (110, 105)), (4, (105, 99)), (4, (104, 97)), (4, (103, 32)), (4, (101, 97)), (4, (100, 32)), (4, (99, 111)), (4, (97, 109)), (4, (85, 110)), (4, (32, 119)), (4, (32, 111)), (4, (32, 102)), (4, (32, 85)), (3, (118, 101)), (3, (116, 115)), (3, (116, 114)), (3, (116, 111)), (3, (114, 116)), (3, (114, 115)), (3, (114, 101)), (3, (111, 102)), (3, (111, 32)), (3, (108, 108)), (

In [5]:
top_pair = max(counts, key=counts.get)
top_pair

(101, 32)

In [6]:
def merge(tokens, pair, num):
    newtokens = []
    i = 0
    n = len(tokens)
    while i < n:
        if i + 1 < n and tokens[i] == pair[0] and tokens[i+1] == pair[1]:
            newtokens.append(num)
            i += 2
        else:
            newtokens.append(tokens[i])
            i += 1
    return newtokens

#test
new = merge([1, 2, 3, 1, 2, 1, 2, 3, 4, 2, 3], (1, 2), 256)
new

[256, 3, 256, 256, 3, 4, 2, 3]

In [7]:
def bpe(tokens, num=256):
    bpetokens = tokens
    while True:
        counts = max_pairs(bpetokens)
        if max(counts.values()) < 2: 
            break
        max_pair = max(counts, key=counts.get)
        
        bpetokens = merge(bpetokens, max_pair, num)
        num += 1
    return bpetokens
bpetokens = bpe(tokens)
print(bpetokens, "\nlength:", len(bpetokens))
print(max(bpetokens))

[239, 188, 181, 266, 142, 266, 137, 266, 131, 266, 143, 266, 132, 266, 133, 296, 263, 164, 263, 157, 263, 152, 263, 146, 263, 158, 263, 147, 263, 148, 258, 189, 32, 264, 186, 268, 179, 268, 174, 268, 168, 268, 180, 268, 169, 268, 170, 296, 257, 152, 132, 32, 84, 104, 256, 297, 298, 110, 299, 286, 300, 301, 102, 287, 32, 288, 97, 302, 304, 104, 287, 305, 306, 314, 119, 270, 108, 100, 119, 315, 101, 291, 87, 316, 317, 107, 318, 119, 32, 302, 279, 319, 271, 116, 111, 32, 258, 156, 322, 270, 323, 324, 157, 32, 259, 32, 279, 114, 32, 115, 290, 116, 119, 265, 256, 40, 325, 326, 101, 297, 275, 97, 271, 109, 101, 261, 115, 258, 148, 108, 300, 256, 327, 293, 119, 99, 104, 265, 95, 271, 329, 97, 292, 278, 286, 272, 330, 105, 319, 116, 63, 41, 291, 66, 117, 323, 256, 332, 316, 98, 286, 327, 101, 276, 288, 100, 105, 118, 293, 304, 262, 279, 115, 274, 45, 112, 97, 103, 256, 284, 256, 83, 116, 274, 265, 100, 32, 112, 108, 117, 260, 105, 305, 333, 122, 295, 260, 306, 322, 108, 101, 109, 295, 116, 265

This makes a nice in between. Instead of a 256 vocab size, we are using a 335 vocab size. But at the same time, we were able to shrink the length of the sequence from 553 -> 300 and shrink our token array from 616 -> 300

In [8]:
# longer text
text = """Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.Diversity and Inherent ComplexityThe Unicode CodespaceCodespace AllocationScriptsUsage FrequencyEncodingsUTF-8UTF-16Combining MarksCanonical EquivalenceNormalization FormsGrapheme ClustersAnd More…Diversity and Inherent ComplexityAs soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!The Unicode CodespaceLet’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.Codespace AllocationTo get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.Map of the Unicode codespace (click to zoom)White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.(In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.ScriptsLet’s zoom in on the first three planes, since that’s where the action is:Map of scripts in Unicode planes 0–2 (click to zoom)This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).Usage FrequencyOne more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.EncodingsWe’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.UTF-8In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:UTF-8 (binary) 	Code point (binary) 	Range0xxxxxxx 	xxxxxxx 	U+0000–U+007F110xxxxx 10yyyyyy 	xxxxxyyyyyy 	U+0080–U+07FF1110xxxx 10yyyyyy 10zzzzzz 	xxxxyyyyyyzzzzzz 	U+0800–U+FFFF11110xxx 10yyyyyy 10zzzzzz 10wwwwww 	xxxyyyyyyzzzzzzwwwwww 	U+10000–U+10FFFFA handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.UTF-16The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:UTF-16 (binary) 	Code point (binary) 	Rangexxxxxxxxxxxxxxxx 	xxxxxxxxxxxxxxxx 	U+0000–U+FFFF110110xxxxxxxxxx 110111yyyyyyyyyy 	xxxxxxxxxxyyyyyyyyyy + 0x10000 	U+10000–U+10FFFFA more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)Combining MarksIn the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓A few other places where dynamic character composition shows up in Unicode:Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.A Hebrew example, with niqqud:	אֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹדNormal writing (no niqqud):	את דלתי הזיז הניע, קטב לשכתי ישודDevanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “​ि” = “हि” (“h” + “i” = “hi”).Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”).Canonical EquivalenceIn Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.For example, the Vietnamese letter “ệ” can be expressed in five different ways:Fully precomposed: U+1EC7 “ệ”Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂”Partially precomposed: U+00EA “ê” + U+0323 “◌̣”Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂”Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣”Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!Normalization FormsTo address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.Grapheme ClustersAs we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.And More…There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.Further reading:The Unicode StandardUTF-8 Everywhere ManifestoDark corners of Unicode by EeveeICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related thingsPython 3 Unicode HowtoGoogle Noto Fonts—set of fonts intended to cover all assigned code points"""
tokens = list(text.encode("utf-8"))
bpetokens = bpe(tokens)

print("length:", len(text))
print("length:", len(tokens))
print("length:", len(bpetokens))
print("max token:", max(bpetokens))

length: 23110
length: 24372
length: 5362
max token: 2133


BPE optimization: 

original text - length: 23110

utf8 - length: 24372

bpe - length: 5362, max token: 2133

In [9]:
# put everything together + keep track of merges
def max_pairs(tokens):
    counts = {}
    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(tokens, pair, num):
    newtokens = []
    i = 0
    n = len(tokens)
    while i < n:
        if i + 1 < n and tokens[i] == pair[0] and tokens[i+1] == pair[1]:
            newtokens.append(num)
            i += 2
        else:
            newtokens.append(tokens[i])
            i += 1
    return newtokens

def bpe(tokens, num=256, vocab_size=1e6):
    bpetokens = tokens
    merges = {}

    while num < vocab_size:
        counts = max_pairs(bpetokens)
        if max(counts.values()) < 2: 
            break
        max_pair = max(counts, key=counts.get)
        merges[max_pair] = num
        bpetokens = merge(bpetokens, max_pair, num)
        # print(f"Merged {max_pair} into {num}")
        num += 1
    # print(f"Vocab size: {len(merges) + 256}")
    return bpetokens, merges

bpetokens, merges = bpe(tokens, vocab_size = 400)


In [10]:
bpetokens, merges = bpe(tokens, vocab_size = 400)
vocab = {idx : bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
    
print(f"Vocab size: {len(vocab.items())}")

Vocab size: 400


In [11]:
def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text
decode([399])

'diacritic'

One thing to note is that certain bytes actually cause errors because the binary representation of the byte does not follow UTF-8 coding standards. take for example 128. Therefore, I am replacing non-decodable characters into an unidentifyable character byte. 

In [12]:
merges

{(101, 32): 256,
 (105, 110): 257,
 (115, 32): 258,
 (116, 104): 259,
 (101, 114): 260,
 (99, 111): 261,
 (116, 32): 262,
 (226, 128): 263,
 (44, 32): 264,
 (97, 110): 265,
 (111, 114): 266,
 (100, 32): 267,
 (97, 114): 268,
 (101, 110): 269,
 (257, 103): 270,
 (261, 100): 271,
 (121, 32): 272,
 (97, 108): 273,
 (259, 256): 274,
 (111, 110): 275,
 (111, 32): 276,
 (116, 105): 277,
 (97, 99): 278,
 (101, 115): 279,
 (111, 102): 280,
 (271, 256): 281,
 (46, 32): 282,
 (112, 111): 283,
 (280, 32): 284,
 (270, 32): 285,
 (101, 267): 286,
 (116, 260): 287,
 (116, 276): 288,
 (257, 32): 289,
 (97, 32): 290,
 (266, 32): 291,
 (115, 116): 292,
 (111, 117): 293,
 (120, 120): 294,
 (99, 104): 295,
 (101, 120): 296,
 (101, 258): 297,
 (263, 156): 298,
 (263, 157): 299,
 (261, 109): 300,
 (265, 267): 301,
 (112, 108): 302,
 (105, 116): 303,
 (283, 257): 304,
 (114, 101): 305,
 (273, 108): 306,
 (110, 105): 307,
 (97, 116): 308,
 (260, 32): 309,
 (263, 153): 310,
 (105, 258): 311,
 (281, 304): 312,

In [14]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) > 1:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break # nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
        
    return tokens
print(encode("junction"))
encode("diacritic")

[106, 117, 110, 99, 317]


[399]

Now lets move on to REGEX patterns
A regex pattern splits the string based on punctuation, spaces, etc. The purpose of regex patterns is to stop specific merges between letters and numbers and etc. By preventing specific merges, we can force the model to look at sentences the same way we are. For example, without regex the model might pair up "n fac" in the phrase "in fact" even though we wouldn't want the model to contain tokenization that doesn't really make sense. 

Flaw:
Sometimes the tokenization works for but not uppercase
for example 
1. how's -> how, 's
2. HOW'S -> HOW'S

In [6]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Hello! How are you. THEY C'sAS INSANE"))

['Hello', '!', ' How', ' are', ' you', '.', ' THEY', ' C', "'s", 'AS', ' INSANE']


Another pattern is the space + word. With this combination, we can keep consistency and everything other than the first line of a sentence 

Here is another real world example (regarding code)

In [7]:
example = """
for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
"""
print(re.findall(gpt2pat, example))

['\n', 'for', ' i', ' in', ' range', '(', '1', ',', ' 101', '):', '\n   ', ' if', ' i', ' %', ' 3', ' ==', ' 0', ' and', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'FizzBuzz', '")', '\n   ', ' elif', ' i', ' %', ' 3', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Fizz', '")', '\n   ', ' elif', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Buzz', '")', '\n   ', ' else', ':', '\n       ', ' print', '(', 'i', ')', '\n']


OpenAI maintains additional rules apart from chunking and bpe. OpenAI never released the training code for GPT-2’s tokenizer—only inference code. So we don’t know the exact training procedure. it wasn’t just “chunk then run vanilla BPE,” because e.g. the “spaces don’t merge” behavior implies extra rules beyond plain BPE.

GPT4 Differences:

Case-insensitive handling of contractions ('s, 'd, 'm, etc.).\
Different whitespace handling (details are fiddly).\
Numbers: matches only 1–3 digits at a time, avoiding very long numeric tokens.\
Motivations aren’t documented; we just have the pattern.

Also: GPT-2 ~ 50k tokens → GPT-4 ~ 100k (cl100k).

Overall pipeline (same as us) used in GPT: byte-encode → BPE-encode → BPE-decode → byte-decode.

In [8]:
!pip install tiktoken # added for colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.0 MB/s eta 0:00:00 0:00:01


In [9]:
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

--2025-09-02 21:32:10--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 57.150.104.161
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|57.150.104.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [application/octet-stream]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>] 445.62K  1.80MB/s    in 0.2s    

2025-09-02 21:32:11 (1.80 MB/s) - ‘vocab.bpe’ saved [456318/456318]

--2025-09-02 21:32:11--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 57.150.104.161
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|57.150.104.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Sav

In [10]:
import os, json

with open('encoder.json', 'r') as f:
    encoder = json.load(f) # <--- ~equivalent to our "vocab"

with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
# ^---- ~equivalent to our "merges"

In [11]:
len(encoder)

50257

### **SPECIAL TOKENS**
As you can see the standard number of tokens is 256 and the number of merges is 50k. So what is this extra 1 special token?\
50000 + 256 + 1 = 50257

In [14]:
list(encoder.items())[-1]

('<|endoftext|>', 50256)

When working with multiple documents, we place end of text in order to tell the model that the previous document is unrelated to the next document\
We also get special case handling for special tokens. Whenever we see special tokens, we don't go through typical byte pair encoding but instead go through special instructions.
In prompt based models, we also have the \<im_start\> and \<im_end\> tokens

<secret text hahahahasdhflkjhsdkjfhasdkjfh>